<center><h1> Case Study 2</h1></center>
<center><h3> Week 2 (out of 5)</h3></center>

**Author(s):**
1. Robin Fu (robin.fu@emory.edu)
 
**Data Source**: W.C. Hunter and M.B. Walker (1996), [“*The Cultural Affinity Hypothesis and Mortgage Lending Decisions*,”](https://link.springer.com/article/10.1007/BF00174551) Journal of Real Estate Finance and Economics 13, 57-70.
 
**Book**: [Introductory Econometrics: A Modern Approach](https://economics.ut.ac.ir/documents/3030266/14100645/Jeffrey_M._Wooldridge_Introductory_Econometrics_A_Modern_Approach__2012.pdf) by Jeffrey Wooldridge

**Data Description**: ```http://fmwww.bc.edu/ec-p/data/wooldridge/loanapp.dta```

```
  Obs:  1989

  1. occ                       occupancy
  2. loanamt                   loan amt in thousands
  3. action                    type of action taken
  4. msa                       msa number of property
  5. suffolk                   =1 if property in Suffolk County
  6. race                      race of applicant
  7. gender                    gender of applicant
  8. appinc                    applicant income, $1000s
  9. typur                     type of purchaser of loan
 10. unit                      number of units in property
 11. married                   =1 if applicant married
 12. dep                       number of dependents
 13. emp                       years employed in line of work
 14. yjob                      years at this job
 15. self                      self-employment dummy
 16. atotinc                   total monthly income
 17. cototinc                  coapp total monthly income
 18. hexp                      propose housing expense
 19. price                     purchase price
 20. other                     other financing, $1000s
 21. liq                       liquid assets
 22. rep                       no. of credit reports
 23. gdlin                     credit history meets guidelines
 24. lines                     no. of credit lines on reports
 25. mortg                     credit history on mortgage paym
 26. cons                      credit history on consumer stuf
 27. pubrec                    =1 if filed bankruptcy
 28. hrat                      housing exp, % total inccome
 29. obrat                     other oblgs,  % total income
 30. fixadj                    fixed or adjustable rate?
 31. term                      term of loan in months
 32. apr                       appraised value
 33. prop                      type of property
 34. inss                      PMI sought
 35. inson                     PMI approved
 36. gift                      gift as down payment
 37. cosign                    is there a cosigner
 38. unver                     unverifiable info
 39. review                    number of times reviewed
 40. netw                      net worth
 41. unem                      unemployment rate by industry
 42. min30                     =1 if minority pop. > 30%
 43. bd                        =1 if boarded-up val > MSA med
 44. mi                        =1 if tract inc > MSA median
 45. old                       =1 if applic age > MSA median
 46. vr                        =1 if tract vac rte > MSA med
 47. sch                       =1 if > 12 years schooling
 48. black                     =1 if applicant black
 49. hispan                    =1 if applicant Hispanic
 50. male                      =1 if applicant male
 51. reject                    =1 if action == 3
 52. approve                   =1 if action == 1 or 2
 53. mortno                    no mortgage history
 54. mortperf                  no late mort. payments
 55. mortlat1                  one or two late payments
 56. mortlat2                  > 2 late payments
 57. chist                     =0 if accnts deliq. >= 60 days
 58. multi                     =1 if two or more units
 59. loanprc                   amt/price
 60. thick                     =1 if rep > 2
 61. white                     =1 if applicant white
 62. obwhte                    obrat*awhite
 ```

1. [5 points] After loading the data set into a Pandas DataFrame, use the approaches described [here](https://stackoverflow.com/questions/50326157/create-dummy-variables-for-interdependent-categories-in-pandas]) to create dummies for the interdependent race categories `black`, `hispan`, `white` and gender category `male`. **Hint:** You should have generated a total of 6 new dummy variables.

In [1]:
import pandas as pd
import numpy as np
import patsy
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import confusion_matrix

#Turning off pandas and numpy warnings
pd.options.mode.chained_assignment = None  # default='warn'
np.seterr(all = 'ignore')

#Loading data and dropping NA rows, esp for male column
df = pd.read_stata('http://fmwww.bc.edu/ec-p/data/wooldridge/loanapp.dta')
df = df.dropna()

#Creating 6 dummy variables by race and gender
dummies = pd.get_dummies(df.race.astype(str) + df.male.astype(str))
dummies.columns = ['black_female','black_male',
                   'hispan_female','hispan_male',
                   'white_female','white_male']
df[dummies.columns] = dummies

2. [5 points] Rename the previously 6 newly created dummy variables accordingly, i.e., `black_male` equals 1 for a black male applicant and 0 otherwise, `white_female` equals 1 for a white female applicant and 0 otherwise, etc.

Columns have been properly named above in part 1. during their creation.

3. You are interested in building a model that accurately predict loan rejection (`rejection`) based on various applicants' features such as `loanamt`, `appinc`, `unit`, `married`, `dep`, `emp`, `yjob`, `self`, `atotinc`, `cototinc`, `hexp`, `price`, `other`, `liq`, `rep`, `gdlin`, `lines`, `mortg`, `cons`, `pubrec`, `hrat`, `obrat`, `fixadj`, `term`, `apr`, `gift`, `cosign`, `netw`, `uem`, `min30`, `bd`, `mi`, `old`, `vr`, `sch`, `mortno`, `chist`, `multi`, `loanprc`, `thick`, and 5 of the 6 newly created race-gender dummies, i.e., make the `white_male` the base category.
    1. [5 points] Replace *all* the proposed features measured in USD with their natural logarithm provided their values are strictly positive. If they are zero instead, then leave them as such.
    2. [5 points] *Add* demeaned squared terms of *all* features measured in units of time to your data frame, e.g., `lemp2` where $\texttt{lemp2}=(\ln(\texttt{emp})-\overline{ln(\texttt{emp}})^2$ where $\overline{ln(\texttt{emp}}$ represents the sample average of the `lemp2` variable. **Note**: You do not need to do this for the remaining features that are not measured in units of time.
    3. [10 points] *Add* to your data frame _all_ products between the 5 race-gender dummy variables and the other **non-dummy** features in the data frame so far (including those you created in point B. above).

In [2]:
#Subsetting data set and setting white_male to be base category
df = df[['reject','loanamt','appinc','unit','married','dep','emp','yjob','self','atotinc','cototinc','hexp','price','other',
          'liq','rep','gdlin','lines','mortg','cons','pubrec','hrat','obrat','fixadj','term','apr','gift','cosign','netw','unem',
          'min30','bd','mi','old','vr','sch','mortno','chist','multi','loanprc','thick','black_female','black_male',
          'hispan_female','hispan_male','white_female']]

#Part A: Replacing features in USD with natural log of values
usd = ['loanamt','appinc','atotinc','cototinc','hexp','price','other','liq','netw','apr']
df[usd] = df[usd].where(df <= 0, np.log(df))

#Part B: adding demeaned squared terms to all features in units of time
time = ['emp','yjob','term']
time2 = [var + '2' for var in time]
df[time2] = df[time].sub(df[time].mean()) ** 2

#Part C:
#Creating demeaned variables
tmp = df.columns[1:]
demeaned = []
for x in tmp:
    df[x + '_dmean'] = df[x] - df[x].mean()
    demeaned.append(x + '_dmean')

In [3]:
#Creating interacted features
dumvars = [x + '_dmean' for x in list(dummies)][:-1]
for dum in dumvars:
    for var in demeaned:
        df[dum + ':' + var] = df[dum] * df[var]

In [4]:
#Creating design matrices
f = 'reject ~ -1 +' + '+'.join(df.columns[1:].difference(demeaned, sort = False))
y, X = patsy.dmatrices(f, data = df, return_type = 'dataframe')

In [5]:
#Create Standard Scaler Object
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#Create standardized X
X_st = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

#Create train and test sets
from sklearn.model_selection import train_test_split
Xst_train, Xst_test, y_train, y_test = train_test_split(X_st, y, test_size=0.2, random_state=42)

#Create K-Fold Cross Validation Object
from sklearn.model_selection import KFold
fold = KFold(n_splits=5, random_state=42, shuffle=True)

#Data Frame to store scores
scores = pd.DataFrame(columns = ['score', 'b_score','num_regs','b_regs'])

4. [20 points] Perform a __Ridge__ Logistic Regression of your model and choose the necessary hyperparameter via a 5-fold cross-validation. Report your logistic score and the confusion matrix based on a validation test made up of 20% of the original sample (use 42 as the seed). **Note**: You can do this via the `LogisticRegressionCV` function from the `sklearn.linear_model` subpackage by choosing a suitable set for the `Cs` and `l1_ratios` accordingly.

In [6]:
RidgeCV = LogisticRegressionCV(
    Cs=list(np.linspace(0.01,20,60,endpoint=True)) #this corresponds to 1/lambda above
    ,penalty='l2'
    ,scoring='accuracy' #proportion of main diag of confusion matrix
    ,cv=fold
    ,random_state=42
    ,max_iter=10000
    ,fit_intercept=True
    ,solver='saga' 
    ,tol=10
)
RLogit_cv = RidgeCV.fit(Xst_train, y_train.values.ravel())

print(RLogit_cv.C_,'\n')
print(RLogit_cv.score(Xst_test, y_test),'\n')
scores.loc['Ridge','score'] = RLogit_cv.score(Xst_test, y_test)
print(confusion_matrix(y_test, RLogit_cv.predict(Xst_test)))
scores.loc['Ridge','num_regs'] = pd.DataFrame(RLogit_cv.coef_).any().sum()

[18.3059322] 

0.8258426966292135 

[[289  30]
 [ 32   5]]


We note that we have a very imbalanced data set. As such, we consider an alternative scoring method using the "balanced_accuracy" scorer. This method is designed to handle unbalanced data sets and is defined as the average of recall based on each class of the outcome. 

In [7]:
df['reject'].value_counts()

0.0    1553
1.0     225
Name: reject, dtype: int64

In [8]:
RidgeCV = LogisticRegressionCV(
    Cs=list(np.linspace(0.001,10,60,endpoint=True)) #this corresponds to 1/lambda above
    ,penalty='l2'
    ,scoring='balanced_accuracy' #proportion of main diag of confusion matrix
    ,cv=fold
    ,random_state=42
    ,max_iter=10000
    ,fit_intercept=True
    ,solver='saga' 
    ,tol=10
)
RLogit_cv = RidgeCV.fit(Xst_train, y_train.values.ravel())

print(RLogit_cv.C_,'\n')
print(RLogit_cv.score(Xst_test, y_test),'\n')
scores.loc['Ridge','b_score'] = RLogit_cv.score(Xst_test, y_test)
print(confusion_matrix(y_test, RLogit_cv.predict(Xst_test)))
scores.loc['Ridge','b_regs'] = pd.DataFrame(RLogit_cv.coef_).any().sum()

[0.001] 

0.5646022197746337 

[[274  45]
 [ 27  10]]


5. [20 points] Perform a __Lasso__ Logistic Regression of your model and choose the necessary hyperparameter via a 5-fold cross-validation. Report your logistic score and the confusion matrix based on a validation test made up of 20% of the original sample (use 42 as the seed). **Note:** You can do this via the `LogisticRegressionCV` function from the `sklearn.linear_model` subpackage by choosing a suitable set for the `Cs` and `l1_ratios` accordingly.

In [9]:
LassoCV = LogisticRegressionCV(
    Cs=list(np.linspace(0.001,10,20,endpoint=True)) #this corresponds to 1/lambda above
    ,penalty='l1'
    ,scoring='accuracy' #proportion of main diag of confusion matrix
    ,cv=fold
    ,random_state=42
    ,max_iter=10000
    ,fit_intercept=True
    ,solver='saga' 
    ,tol=10
)
LLogit_cv = LassoCV.fit(Xst_train, y_train.values.ravel())

print(LLogit_cv.C_,'\n')
print(LLogit_cv.score(Xst_test, y_test),'\n')
scores.loc['Lasso','score'] = LLogit_cv.score(Xst_test, y_test)
print(confusion_matrix(y_test, LLogit_cv.predict(Xst_test)))
scores.loc['Lasso','num_regs'] = pd.DataFrame(LLogit_cv.coef_).any().sum()

[0.001] 

0.8960674157303371 

[[319   0]
 [ 37   0]]


In [10]:
LassoCV = LogisticRegressionCV(
    Cs=list(np.linspace(0.001,3,50,endpoint=True))
    ,penalty='l1'
    ,scoring='balanced_accuracy' #proportion of main diag of confusion matrix
    ,cv=fold
    ,random_state=42
    ,max_iter=10000
    ,fit_intercept=True
    ,solver='saga' 
    ,tol=10
)
LLogit_cv = LassoCV.fit(Xst_train, y_train.values.ravel())

print(LLogit_cv.C_,'\n')
print(LLogit_cv.score(Xst_test, y_test),'\n')
scores.loc['Lasso','b_score'] = LLogit_cv.score(Xst_test, y_test)
print(confusion_matrix(y_test, LLogit_cv.predict(Xst_test)))
scores.loc['Lasso','b_regs'] = pd.DataFrame(LLogit_cv.coef_).any().sum()

[0.18461224] 

0.5636278912140981 

[[282  37]
 [ 28   9]]


4. [20 points] Perform an __Elastic Net__ Logistic Regression of your model and choose the necessary hyperparameter via a 5-fold cross-validation. Report your logistic score and the confusion matrix based on a validation test made up of 20% of the original sample (use 42 as the seed).

In [11]:
ElastCV = LogisticRegressionCV(
    Cs=list(np.linspace(0.001,10,30,endpoint=True))
    ,penalty='elasticnet'
    ,l1_ratios=np.linspace(0.1,0.9,5,endpoint=True) 
    ,scoring='accuracy' #proportion of main diag of confusion matrix
    ,cv=fold
    ,random_state=42
    ,max_iter=10000
    ,fit_intercept=True
    ,solver='saga' 
    ,tol=10
)

ELogit_cv = ElastCV.fit(Xst_train, y_train.values.ravel())

print(ELogit_cv.C_,'\n')
print(ELogit_cv.l1_ratio_,'\n')
print(ELogit_cv.score(Xst_test, y_test),'\n')
scores.loc['Elastic Net','score'] = ELogit_cv.score(Xst_test, y_test)
print(confusion_matrix(y_test, ELogit_cv.predict(Xst_test)))
scores.loc['Elastic Net','num_regs'] = pd.DataFrame(ELogit_cv.coef_).any().sum()

[0.001] 

[0.1] 

0.851123595505618 

[[300  19]
 [ 34   3]]


Note that for the above regression, we could lower the Cs to 0.0001 to increase the score. However, this result will be effectively identical to the Lasso Logistic Regression above. More explanation on this below.

In [12]:
ElastCV = LogisticRegressionCV(
    Cs=list(np.linspace(0.0001,1,30,endpoint=True))
    ,penalty='elasticnet'
    ,l1_ratios=np.linspace(0.1,0.9,10,endpoint=True) 
    ,scoring='balanced_accuracy'
    ,cv=fold
    ,random_state=42
    ,max_iter=10000
    ,fit_intercept=True
    ,solver='saga' 
    ,tol=10
)

ELogit_cv = ElastCV.fit(Xst_train, y_train.values.ravel())

print(ELogit_cv.C_,'\n')
print(ELogit_cv.l1_ratio_,'\n')
print(ELogit_cv.score(Xst_test, y_test),'\n')
scores.loc['Elastic Net','b_score'] = ELogit_cv.score(Xst_test, y_test)
print(confusion_matrix(y_test, ELogit_cv.predict(Xst_test)))
scores.loc['Elastic Net','b_regs'] = pd.DataFrame(ELogit_cv.coef_).any().sum()

[0.10353793] 

[0.36666667] 

0.5604930949758536 

[[280  39]
 [ 28   9]]


5. [10 points] What machine among the three you built would you use and why?

Consider the scores for the various machines below:

In [13]:
scores

,score,b_score,num_regs,b_regs
Ridge,0.825843,0.564602,278,278
Lasso,0.896067,0.563628,0,278
Elastic Net,0.851124,0.560493,152,278


Comparing the scores of the various machines, we also note that the Lasso Logistic Regression has the highest score (i.e. best prediction power) when judged based on pure accuracy. However, the Lasso Logistic Regression selects 0 regressors. This is due to the fact that our data is very imbalanced. Looking above at the confusion matrices, we note that this machine never predicts any rejections which produces its high score. A similar result would be seen with the Elastic Net if we were to allow the C value to go lower. As such, it is questionable whether the score based on accuracy alone can be trusted given the imbalance in our data set.

That conclusion brings us to consider the balanced_score method. While this method may decrease the accuracy of the model, the score itself may be considered for the sake of thoroughness. In this instance, the Ridge Regression has the highest balanced_score. The Lasso Regression follows this and is followed by the Elastic Net. However, the differences among all of these machines are fairly small. It is also important to note that all models select all 278 regressors in the balanced accuracy comparison. As such, while worth the consideration, balanced_score did not provide much argument in favor of any regression. Thus, we will still focus on the pure accuracy score models.

We propose again that the motivation behind the machine is to find one with powerful prediction power while selecting the optimal and ideally more parsimonious model. Given these above considerations, we propose that the __*Elastic Net*__ is the best machine for prediction for several reasons. With proper limiting to the C value and selection of l1_ratio, the Elastic Net provides a good balance betweent he Ridge and Lasso. This is evident in both its pure accuracy score and the number of regressors it selects. Importantly, it helps select fewer regressors than the Ridge, one of the goals behind our machine, while maintaining higher prediction power. The Elastic Net machine also produces a model that still predicts some outcomes to be rejections while selecting regressors, two key failures of the Lasso machine. As such, the Elastic Net is the optimal machine to use.